<h3 style="text-align: center;">ESCUELA TÉCNICA SUPERIOR DE INGENIERÍA INFORMÁTICA</h3>
<h3 style="text-align: center;">UNIVERSIDAD DE SEVILLA</h3>
<h3 style="text-align: center;">MATEMÁTICA APLICADA A SISTEMAS DE INFORMACIÓN</h3>
<h3>Segunda práctica de laboratorio</h3>
<p>En esta práctica estudiaremos métodos de optimización sin restricciones de funciones de varias variables. Es decir partimos del problema:</p>
<p>$$\min_{x\in D}f(x)$$</p>
<p>Gracias al teorema de Weierstrass sabemos que en un conjunto $D$ cerrado y acotado siempre se alcanza el máximo y el mínimo. Hemos estudiado métodos analíticos que nos permiten calcular dichos óptimos. En esta práctica nos centraremos en los métodos numéricos para la resolución de dichos problemas de óptimización de funciones de varias variables.</p>
<p>Empezaremos estudiando un caso analítico.</p>
<p>Supongamos que estamos interesados en calcular el mínimo de la función:</p>
<p>$$f(x,y)=3x^2+y^2-x^4-12$$</p>

In [ ]:
x,y=var('x,y')

In [ ]:
f(x,y)=3*x^2+y^2-x^4-12

In [ ]:
plot3d(f,(x,-3,3),(y,-3,3))

<p>En la gráfica no se distinguen muy bien los máximos, mínimos y puntos de inflexión:</p>
<p>Calculamos el conjuto de puntos críticos de la función $f(x,y)$</p>

In [ ]:
gradiente=f(x,y).gradient()

derpar0=solve(list(gradiente),x,y)

lptc=[(sol[0].rhs(),sol[1].rhs()) for sol in derpar0]

puntos_criticos=[]
for ptc in lptc:
    if (ptc[0].n()).imag()==0 and (ptc[1].n()).imag()==0:
        puntos_criticos.append(ptc)
        
print ('lista de puntos críticos:')
#print (puntos_criticos)
show(puntos_criticos)

In [ ]:
gradiente

In [ ]:
derpar0

In [ ]:
lptc

In [ ]:
hess=f.hessian()
def hessiano(pto):
    return hess(x=pto[0],y=pto[1])
    
print ('matriz hessiana:')    
show(hess)

<p>Hemos definido un proceso que nos permite sustituir las coordenadas de un punto en la matriz hessiana, ahora queremos ver si las matrices hessianas en nuestros puntos críticos son definidas positivas, definidas negativas o indefinidas en signo. Para ello:</p>

In [ ]:
matriz_hessiana=matrix(RDF,((f.hessian()).substitute(x=puntos_criticos[0][0],y=puntos_criticos[0][1])))

In [ ]:
(matriz_hessiana).is_positive_definite()

In [ ]:
for j in range(len(puntos_criticos)):
    print ('punto critico=', j+1)
    for i in range((hessiano(puntos_criticos[j])).nrows()):
        print ('menor de orden ',i+1,'=', (hessiano(puntos_criticos[j])[0:i+1,0:i+1]).det())

<p>Acabamos de ver que el primer punto crítico de nuestra lista es un mínimo local y los dos últimos son puntos de silla</p>

<p>Si nos fjamos en la gráfica de la función, el mínimo en $(0,0)$ es un mínimo local.</p>

<h3>Método del gradiente</h3>
<p>Vamos ahora a utilizar el método del gradiente, en primer lugar con búsqueda lineal exacta y después usando la regla de Armijo, en primer lugar necesitaremos introducir una nueva variable, que llamamos $a$.</p>

In [ ]:
a=var('a')

In [ ]:
f(x,y)=3*x^2+y^2-x^4-12

In [ ]:
punto_inicio=vector([1/3,1])
gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])  
phi(a)=f(punto_inicio[0]+a*gradiente0[0],punto_inicio[1]+a*gradiente0[1])

<p>Si calculamos de forma exacta las raíces, con la orden solve</p>

In [ ]:
soluciones=solve(phi.diff(a)==0,a)

In [ ]:
soluciones[0].rhs().n()

<p>Para evitar problemas en el cálculo de las raíces (como que pueda considerarlas complejas, siendo reales), nos interesa calcularlas numéricamente con la orden find_root. El principal problema de esta orden es que tenemos que darle un intervalo de búsqueda y en principio no sabemos cómo de lejos puede estar el óptimo.</p>

In [ ]:
find_root(phi.diff(a)==0,0,1)

<p>Si dibujamos la función $\varphi(a)$ podemos apreciar que en $x=0.91001$ no tenemos un mínimo, hay un máximo, y de hecho si tomamos este valor de $a$, la función $f$ será mayor en el nuevo punto respecto del punto de partida</p>

In [ ]:
plot(phi,(a,0,1))

<p>Por todo esto debemos de poner la condición de que se produzca (al menos) descenso, o para comprobar que es un mínimo con la condición de la derivada de segundo orden.</p>

In [ ]:
phi.diff(a,2)(a=find_root(phi.diff(a)==0,0,1))

In [ ]:
find_root(phi.diff(a)==0,0,0.999*0.9100101478813016)

In [ ]:
phi.diff(a,2)(a=find_root(phi.diff(a)==0,0,0.999*0.9100101478813016))

In [ ]:
punto_inicio=vector([1/3,1.])
punto_antiguo=vector([10^3,10^3]);error=10^(-3);k=1;
gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])
while (gradiente0).norm()>error:  
    print ('iter.=',k,'gradiente=',gradiente0[0].n(),gradiente0[1].n())
    phi(a)=f(punto_inicio[0]+a*gradiente0[0],punto_inicio[1]+a*gradiente0[1])
    valora=find_root(phi.diff(a)==0,0,5/(gradiente0).norm())
    while phi.diff(a,2)(a=valora)<0:
        valora=find_root(phi.diff(a)==0,0,0.999*valora)
        print ('entra','k=',k)
    print ('valor a=',valora)
    punto_antiguo=punto_inicio
    punto_inicio=punto_inicio+valora*gradiente0
    gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])
    print ('x=',punto_inicio[0].n(),'y=',punto_inicio[1].n())
    k=k+1

<p>Hemos visto que el método del gradiente posee una alternativa, menos costosa computacionalmente, la regla de Armijo, que en términos de $\varphi(a)$, requiere que</p>
<p>$$\varphi(\alpha)\leq\varphi(0)+\epsilon\alpha\varphi'(0)$$</p>

In [ ]:
punto_inicio=vector([1/3,1.])
punto_antiguo=vector([10^3,10^3]);error=10^(-3)
epsilon=1/5;beta=0.5;s=1;m=0;k=1
gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])
while (gradiente0).norm()>error:
    phi(a)=f(punto_inicio[0]+a*gradiente0[0],punto_inicio[1]+a*gradiente0[1])
    alpha=beta^m*s 
    if phi(alpha)<phi(0)+epsilon*alpha*((phi.diff(a))(a=0)):
        punto_antiguo=punto_inicio
        punto_inicio=punto_inicio+alpha*gradiente0
        print ('iter.=',k,'x=',punto_inicio[0].n(),'y=',punto_inicio[1].n())
        print ('error=',(punto_inicio-punto_antiguo).norm())
        m=0
        k=k+1
    else:
        m=m+1
    gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])

<h3>El método de Newton</h3>
<p>Sabemos que hay un método de segundo orden, el método de Newton, donde en cada iteración se calcula</p>
<p>$$x^{k+1}=x^k-[Hf(x^k)]^{-1}\nabla f(x^k)$$</p>
<p>la dirección de Newton $-[Hf(x^k)]^{-1}\nabla f(x^k)$ no es, en general, de descenso, necesitamos que la matriz hessiana $Hf(x^k)$ sea definida positiva.</p>

In [ ]:
f(x,y)=3*x^2+y^2-x^4-12

In [ ]:
punto_inicio=vector([1/3,1.])
hessiano0=matrix(RDF,(f.hessian()).substitute(x=punto_inicio[0],y=punto_inicio[1]))
hessiano0.is_positive_definite()

In [ ]:
gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])
punto_inicio=punto_inicio+(hessiano0)^(-1)*gradiente0

In [ ]:
punto_inicio

In [ ]:
gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])
gradiente0.norm()

In [ ]:
hessiano0=matrix(RDF,(f.hessian()).substitute(x=punto_inicio[0],y=punto_inicio[1]))
hessiano0.is_positive_definite()

In [ ]:
punto_inicio=punto_inicio+(hessiano0)^(-1)*gradiente0

In [ ]:
punto_inicio

In [ ]:
gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])
gradiente0.norm()

<p>En dos iteraciones hemos sidos capaces de poder asegurar $||\nabla f(x^k)||<10^{-2}$</p>
<h3>Método de quasi-Newton</h3>
<p>Sabemos que hay multitud de funciones donde la matriz hessiana no es definida positiva, si consideramos:</p>
<p>$$f(x,y)=16\left(x-\frac{1}{4}\right)^4+3x^2y^2$$</p>
<p>y de punto de inicio $x^0=(\frac{1}{2},1)$, obtenemos que:</p>

In [ ]:
f(x,y)=16*(x-1/4)^4+3*x^2*y^2

In [ ]:
punto_inicio=vector([1/2,1.]);lista1=[punto_inicio]
gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])
gradiente0.norm()
hessiano0=matrix(RDF,(f.hessian()).substitute(x=punto_inicio[0],y=punto_inicio[1]))
hessiano0.is_positive_definite()

<p>Por tanto no tenemos que la dirección de Newton sea una dirección de descenso. Tenemos que modificar la matriz hessiana ligeramente, hacer una perturbación de ella. El algoritmo de quasi-Newton que explicamos está en el texto de Nocedal y Wright.</p>
<p>En la iteración $k$-ésima tenemos que comprobar que la matriz hessiana $H_k$ sea definida positiva, en caso contrario vamos a construir una constante $\tau$, dependiente de $x^k$, de modo que $H_k+\tau I$ sea definida positiva, para ello nos vamos a servir de un proceso iterativo.</p>
<p>Partimos de un valor $\beta>0$, por ejemplo, $\beta=10^{-3}$. Sean $h_{11},\ldots,h_{nn}$ los elementos de la diagonal de $H_k$ </p>
    Si $\min\{h_{11},\ldots,h_{nn}\}\ge 0$, hacemos $\tau=0$. Pero si $\min\{h_{11},\ldots,h_{nn}\}<0$ entonces tomamos $$\tau=-\min\{h_{11},\ldots,h_{nn}\}+\beta$$</p>
<p>Si $H_k+\tau I$ es definda positiva ya está, en caso contrario, se cambia $\tau$, mediante la fórmula $\tau=\max\{2\tau,\beta\}$, hasta que la matriz $H_k+\tau I$ sea definida positiva.</p>

In [ ]:
hessiano0.diagonal()

<p>Todos los elementos de la diagonal son positivos, por tanto no tenemos que cambiar el valor de $\tau$</p>

In [ ]:
beta=10^(-3);t=0

<p>Hemos programado el cálculo de $\tau$, usando una variable temporal hessiano0temp. Se usa copy, ya si hacemos uso de la asignación vinculamos ambas variables y los cambios en el bucle de hessianotemp, se llevarían también a cabo en hessiano0.</p>

In [ ]:
hessiano0temp=copy(hessiano0)
while hessiano0temp.is_positive_definite()==False:
    hessiano0temp=hessiano0+t*identity_matrix(2)
    t=max(2*t,beta)
hessiano0=hessiano0temp

In [ ]:
hessiano0

In [ ]:
hessiano0.is_positive_definite()

In [ ]:
punto_inicio=punto_inicio+(hessiano0)^(-1)*gradiente0
#primera iteracion

In [ ]:
punto_inicio;lista1.append(punto_inicio)

In [ ]:
gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])
gradiente0.norm()

In [ ]:
hessiano0=matrix(RDF,(f.hessian()).substitute(x=punto_inicio[0],y=punto_inicio[1]))
hessiano0.is_positive_definite()

In [ ]:
punto_inicio=punto_inicio+(hessiano0)^(-1)*gradiente0;lista1.append(punto_inicio)
#segunda iteracion

In [ ]:
gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])
gradiente0.norm()

In [ ]:
hessiano0=matrix(RDF,(f.hessian()).substitute(x=punto_inicio[0],y=punto_inicio[1]))
hessiano0.is_positive_definite()

In [ ]:
punto_inicio=punto_inicio+(hessiano0)^(-1)*gradiente0;lista1.append(punto_inicio)
#tercera iteracion
gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])
gradiente0.norm()

In [ ]:
hessiano0=matrix(RDF,(f.hessian()).substitute(x=punto_inicio[0],y=punto_inicio[1]))
hessiano0.is_positive_definite()

In [ ]:
punto_inicio=punto_inicio+(hessiano0)^(-1)*gradiente0;lista1.append(punto_inicio)
#cuarta iteracion
gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])
gradiente0.norm()

In [ ]:
hessiano0=matrix(RDF,(f.hessian()).substitute(x=punto_inicio[0],y=punto_inicio[1]))
hessiano0.is_positive_definite()

In [ ]:
punto_inicio=punto_inicio+(hessiano0)^(-1)*gradiente0;lista1.append(punto_inicio)
#quinta iteracion
gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])
gradiente0.norm()

In [ ]:
hessiano0=matrix(RDF,(f.hessian()).substitute(x=punto_inicio[0],y=punto_inicio[1]))
hessiano0.is_positive_definite()

In [ ]:
punto_inicio=punto_inicio+(hessiano0)^(-1)*gradiente0;lista1.append(punto_inicio)
#sexta iteracion
gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])
gradiente0.norm()

In [ ]:
hessiano0=matrix(RDF,(f.hessian()).substitute(x=punto_inicio[0],y=punto_inicio[1]))
hessiano0.is_positive_definite()

In [ ]:
punto_inicio=punto_inicio+(hessiano0)^(-1)*gradiente0;lista1.append(punto_inicio)
#sexta iteracion
gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])
gradiente0.norm()

<p>Tras 7 iteraciones hemos conseguido que $||\nabla f(x^k)||\leq 10^{-1}$.</p>
<p>Representamos a continuación las iteraciones que hemos llevado a cabo, así como las curvas que representan $|Hf(x,y)|=0$ y por tanto separan las regiones donde el hessiano es una matriz definida positiva o no lo es.</p>

In [ ]:
line(lista1)+point(lista1,color='green',size=20)+plot(sqrt(2/3)*(4*x-1),(x,-2,6),color='red')+plot(-sqrt(2/3)*(4*x-1),(x,-2,6),color='red')

<p>Hemos visto en clase que el método de Newton no tiene convergencia global, depende del punto de inicio. Por ejemplo si partimos del punto $x^0=\left(\frac{1}{4},1\right)$, vamos a aplicarle el mismo método de quasi-Newton.</p>

In [ ]:
punto_inicio=vector([1/4,1]);lista2=[punto_inicio]
gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])
hessiano0=matrix(RDF,(f.hessian()).substitute(x=punto_inicio[0],y=punto_inicio[1]))
hessiano0.is_positive_definite();gradiente0.norm().n()

In [ ]:
hessiano0.diagonal()

In [ ]:
beta=10^(-3);t=0
hessiano0temp=copy(hessiano0)
while hessiano0temp.is_positive_definite()==False:
    hessiano0temp=hessiano0+t*identity_matrix(2)
    t=max(2*t,beta)
hessiano0=copy(hessiano0temp)

In [ ]:
hessiano0.is_positive_definite()

In [ ]:
punto_inicio=punto_inicio+(hessiano0)^(-1)*gradiente0;lista2.append(punto_inicio)
#primera iteracion

In [ ]:
punto_inicio

In [ ]:
gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])
hessiano0=matrix(RDF,(f.hessian()).substitute(x=punto_inicio[0],y=punto_inicio[1]))
hessiano0.is_positive_definite();gradiente0.norm()

In [ ]:
punto_inicio=punto_inicio+(hessiano0)^(-1)*gradiente0;lista2.append(punto_inicio)
#segunda iteracion

In [ ]:
punto_inicio

In [ ]:
gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])
hessiano0=matrix(RDF,(f.hessian()).substitute(x=punto_inicio[0],y=punto_inicio[1]))
hessiano0.is_positive_definite();gradiente0.norm()

In [ ]:
hessiano0.diagonal()

In [ ]:
beta=10^(-3);t=0
hessiano0temp=copy(hessiano0)
while hessiano0temp.is_positive_definite()==False:
    hessiano0temp=hessiano0+t*identity_matrix(2)
    t=max(2*t,beta)
hessiano0=copy(hessiano0temp)

In [ ]:
hessiano0.is_positive_definite()

In [ ]:
punto_inicio=punto_inicio+(hessiano0)^(-1)*gradiente0;lista2.append(punto_inicio);lista2.append(punto_inicio)
#tercera iteracion

In [ ]:
gradiente0=-(f.gradient()).substitute(x=punto_inicio[0],y=punto_inicio[1])
punto_inicio;gradiente0.norm().n()

In [ ]:
line(lista2)+point(lista2,color='green',size=20)+plot(sqrt(2/3)*(4*x-1),(x,-2,6),color='red')+plot(-sqrt(2/3)*(4*x-1),(x,-2,6),color='red')

<p> </p>
<h3>Cuestionario para el alumno</h3>

In [1]:
NUMERO_ALUMNO=1

NOMBRE_FICHERO_EXAMEN = 'Ex_MASI_L2_t.htl'
load('codigo_examinar_html.sage')

if NUMERO_ALUMNO > 0:
    lector_examenes(NOMBRE_FICHERO_EXAMEN,NUMERO_ALUMNO,False)

1.,2.
3.,4.
1.,2.
3.,4.
1. Al menos 12 iteraciones,2. Al menos 86 iteraciones
3. Al menos 52 iteraciones,4. Al menos 27 iteraciones
1.,2.
3.,4. Ninguna de las otras respuestas.
1. Al menos 19 iteraciones,2. Al menos 5 iteraciones
3. Al menos 15 iteraciones,4. Al menos 11 iteraciones
1.,2.
